In [ ]:
!pip install kaggle


In [ ]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"radhikapingale","key":"c0419da54f0724e915234908ed1506f5"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d emmarex/plantdisease -p /content


Dataset URL: https://www.kaggle.com/datasets/emmarex/plantdisease
License(s): unknown
plantdisease.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip /content/plantdisease.zip -d /content/plantvillage_data


Archive:  /content/plantdisease.zip
replace /content/plantvillage_data/PlantVillage/Pepper__bell___Bacterial_spot/0022d6b7-d47c-4ee2-ae9a-392a53f48647___JR_B.Spot 8964.JPG? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!ls /content/plantvillage_data/plantvillage/PlantVillage | head

Pepper__bell___Bacterial_spot
Pepper__bell___healthy
Potato___Early_blight
Potato___healthy
Potato___Late_blight
Tomato_Bacterial_spot
Tomato_Early_blight
Tomato_healthy
Tomato_Late_blight
Tomato_Leaf_Mold


In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split
from glob import glob

source_dataset = "/content/plantvillage_data/plantvillage/PlantVillage"
target_base = "/content/PlantDocBot/data"

train_dir = os.path.join(target_base, "train")
val_dir = os.path.join(target_base, "val")

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Detect class folders
class_folders = [
    d for d in os.listdir(source_dataset)
    if os.path.isdir(os.path.join(source_dataset, d))
]

print("Detected classes:", class_folders)

for cls in class_folders:
    cls_path = os.path.join(source_dataset, cls)

    # Get all images
    images = (
        glob(cls_path + "/*.JPG")
        + glob(cls_path + "/*.jpg")
        + glob(cls_path + "/*.jpeg")
        + glob(cls_path + "/*.png")
    )

    if len(images) == 0:
        print(f"⚠️ No images found in: {cls}")
        continue

    # Split images
    train_imgs, val_imgs = train_test_split(images, test_size=0.2, random_state=42)

    # Create class folders inside train/val
    os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(val_dir, cls), exist_ok=True)

    # Copy files
    for img in train_imgs:
        shutil.copy(img, os.path.join(train_dir, cls))

    for img in val_imgs:
        shutil.copy(img, os.path.join(val_dir, cls))

print("✅ Dataset split completed successfully!")


Detected classes: ['Tomato__Tomato_mosaic_virus', 'Potato___Early_blight', 'Tomato_healthy', 'Tomato_Septoria_leaf_spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Potato___healthy', 'Tomato_Leaf_Mold', 'Tomato__Target_Spot', 'Tomato_Late_blight', 'Tomato_Early_blight', 'Potato___Late_blight', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Pepper__bell___healthy', 'Tomato_Bacterial_spot', 'Pepper__bell___Bacterial_spot']
✅ Dataset split completed successfully!


In [ ]:
!ls /content/PlantDocBot/data/train | head


Pepper__bell___Bacterial_spot
Pepper__bell___healthy
Potato___Early_blight
Potato___healthy
Potato___Late_blight
Tomato_Bacterial_spot
Tomato_Early_blight
Tomato_healthy
Tomato_Late_blight
Tomato_Leaf_Mold


In [ ]:
!ls /content/PlantDocBot/data/val | head


Pepper__bell___Bacterial_spot
Pepper__bell___healthy
Potato___Early_blight
Potato___healthy
Potato___Late_blight
Tomato_Bacterial_spot
Tomato_Early_blight
Tomato_healthy
Tomato_Late_blight
Tomato_Leaf_Mold


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader

train_dir = "/content/PlantDocBot/data/train"
val_dir = "/content/PlantDocBot/data/val"

# Image transforms
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ToTensor()
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
val_dataset = datasets.ImageFolder(val_dir, transform=val_transform)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

num_classes = len(train_dataset.classes)
print("Detected Classes:", train_dataset.classes)


Detected Classes: ['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato__Tomato_mosaic_virus', 'Tomato_healthy']


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model = models.resnet50(weights="IMAGENET1K_V2")

# Replace final FC layer
model.fc = nn.Linear(model.fc.in_features, num_classes)

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)


Using device: cuda


In [ ]:
epochs = 5  # You can increase to 10–15 later for better accuracy

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}] - Loss: {total_loss:.4f}")

print("Training complete!")


Epoch [1/5] - Loss: 202.1985
Epoch [2/5] - Loss: 32.2732
Epoch [3/5] - Loss: 19.2950
Epoch [4/5] - Loss: 12.3399
Epoch [5/5] - Loss: 13.2796
Training complete!


In [ ]:
import os
os.makedirs("/content/PlantDocBot/models", exist_ok=True)
print("Models folder created!")


Models folder created!


In [ ]:
save_path = "/content/PlantDocBot/models/image_model.pth"
torch.save(model.state_dict(), save_path)

print("Model saved at:", save_path)


Model saved at: /content/PlantDocBot/models/image_model.pth


In [ ]:
import torch
from torchvision import transforms, models
from PIL import Image
import os

# Load class names from training folder
train_dir = "/content/PlantDocBot/data/train"
class_names = sorted(os.listdir(train_dir))

# Load the saved model
def load_image_model():
    num_classes = len(class_names)
    model = models.resnet50(weights="IMAGENET1K_V2")
    model.fc = torch.nn.Linear(model.fc.in_features, num_classes)

    model_path = "/content/PlantDocBot/models/image_model.pth"
    model.load_state_dict(torch.load(model_path, map_location="cpu"))
    model.eval()
    return model

image_model = load_image_model()
print("✅ Model loaded successfully!")

# Image preprocessing
img_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Prediction function
def predict_from_image(image_path):
    image = Image.open(image_path).convert("RGB")
    img_tensor = img_transform(image).unsqueeze(0)
    with torch.no_grad():
        outputs = image_model(img_tensor)
        _, predicted = torch.max(outputs, 1)
    return class_names[predicted.item()]


✅ Model loaded successfully!


In [ ]:
from google.colab import files
uploaded = files.upload()

for filename in uploaded.keys():
    prediction = predict_from_image(filename)
    print("Prediction:", prediction)


Saving istockphoto-496662675-1024x1024.jpg to istockphoto-496662675-1024x1024 (1).jpg
Prediction: Tomato_Late_blight


In [ ]:
disease_treatments = {
    "Tomato_Bacterial_spot": "Use copper-based fungicides, remove infected leaves, avoid overhead watering.",
    "Tomato_Early_blight": "Apply fungicides containing chlorothalonil. Practice crop rotation.",
    "Tomato_Late_blight": "Use systemic fungicides like metalaxyl. Remove infected plants completely.",
    "Tomato_Leaf_Mold": "Increase air circulation, apply fungicides like chlorothalonil.",
    "Tomato_healthy": "Plant appears healthy. Continue regular care.",

    "Potato___Early_blight": "Use fungicides like mancozeb or chlorothalonil. Improve soil drainage.",
    "Potato___Late_blight": "Apply copper fungicides. Destroy infected leaves immediately.",
    "Potato___healthy": "Plant is healthy.",

    "Pepper__bell___Bacterial_spot": "Use bactericides like copper. Remove infected plant parts.",
    "Pepper__bell___healthy": "Healthy plant.",

    "Apple___Black_rot": "Prune infected branches. Apply fungicides like captan or thiophanate-methyl.",
    "Apple___healthy": "Tree looks healthy."
}


In [ ]:
def plantdocbot(image_path=None, text=None):
    result = {}

    # If image provided
    if image_path:
        disease = predict_from_image(image_path)
        treatment = disease_treatments.get(disease, "Treatment information not available.")
        result["type"] = "image"
        result["diagnosis"] = disease
        result["treatment"] = treatment

    # If text provided
    if text:
        prediction = predict_from_text(text)
        treatment = disease_treatments.get(prediction, "Treatment information not available.")
        result["type"] = "text"
        result["diagnosis"] = prediction
        result["treatment"] = treatment

    return result


In [ ]:
response = plantdocbot(image_path="istockphoto-496662675-1024x1024.jpg")
print(response)


{'type': 'image', 'diagnosis': 'Tomato_Late_blight', 'treatment': 'Use systemic fungicides like metalaxyl. Remove infected plants completely.'}


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
BASE_DIR = '/content/drive/MyDrive/PlantBot'


Mounted at /content/drive


In [ ]:
from google.colab import files
files.upload()

Saving istockphoto-496662675-1024x1024.jpg to istockphoto-496662675-1024x1024 (2).jpg


{'istockphoto-496662675-1024x1024 (2).jpg': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x01,\x01,\x00\x00\xff\xe1\x00\xb9Exif\x00\x00II*\x00\x08\x00\x00\x00\x04\x00\x0e\x01\x02\x00Z\x00\x00\x00>\x00\x00\x00\x98\x82\x02\x00\t\x00\x00\x00\x98\x00\x00\x00\x1a\x01\x05\x00\x01\x00\x00\x00\xa1\x00\x00\x00\x1b\x01\x05\x00\x01\x00\x00\x00\xa9\x00\x00\x00\x00\x00\x00\x00Single a green leaf of tomato. Isolated on white background. Close-up. Studio photography.boroda003,\x01\x00\x00\x01\x00\x00\x00,\x01\x00\x00\x01\x00\x00\x00\xff\xe1\x05\xdfhttp://ns.adobe.com/xap/1.0/\x00<?xpacket begin="\xef\xbb\xbf" id="W5M0MpCehiHzreSzNTczkc9d"?>\n<x:xmpmeta xmlns:x="adobe:ns:meta/">\n\t<rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">\n\t\t<rdf:Description rdf:about="" xmlns:photoshop="http://ns.adobe.com/photoshop/1.0/" xmlns:Iptc4xmpCore="http://iptc.org/std/Iptc4xmpCore/1.0/xmlns/"   xmlns:GettyImagesGIFT="http://xmp.gettyimages.com/gift/1.0/" xmlns:dc="http://purl.org/dc/elements/1.1/

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle* ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

cp: target '/root/.kaggle/kaggle.json' is not a directory


In [ ]:
from google.colab import files

# Upload your kaggle.json file
files.upload()  # This will open a file browser


Saving kaggle.json to kaggle (2).json


{'kaggle (2).json': b'{"username":"radhikapingale","key":"c0419da54f0724e915234908ed1506f5"}'}

In [ ]:
# Create the .kaggle folder if it doesn't exist
!mkdir -p ~/.kaggle

# Move the uploaded kaggle.json into the folder
!mv kaggle.json ~/.kaggle/

# Set secure permissions
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d emmarex/plantdisease -p /content


Dataset URL: https://www.kaggle.com/datasets/emmarex/plantdisease
License(s): unknown
plantdisease.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip -q /content/plantdisease.zip -d /content/plantvillage_data

replace /content/plantvillage_data/PlantVillage/Pepper__bell___Bacterial_spot/0022d6b7-d47c-4ee2-ae9a-392a53f48647___JR_B.Spot 8964.JPG? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!ls /content/plantvillage_data/plantvillage/PlantVillage | head


Pepper__bell___Bacterial_spot
Pepper__bell___healthy
Potato___Early_blight
Potato___healthy
Potato___Late_blight
Tomato_Bacterial_spot
Tomato_Early_blight
Tomato_healthy
Tomato_Late_blight
Tomato_Leaf_Mold


In [ ]:
import os, shutil, random
from glob import glob

src_root = "/content/plantvillage_data/plantvillage/PlantVillage"
dst_root = "/content/plantvillage_split"

train_dir = os.path.join(dst_root, "train")
val_dir = os.path.join(dst_root, "val")

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

classes = os.listdir(src_root)
split_ratio = 0.8  # 80% train / 20% val

for cls in classes:
    src_cls = os.path.join(src_root, cls)
    images = glob(os.path.join(src_cls, "*.jpg"))

    random.shuffle(images)
    split = int(len(images) * split_ratio)

    train_imgs = images[:split]
    val_imgs = images[split:]

    os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(val_dir, cls), exist_ok=True)

    for img in train_imgs:
        shutil.copy(img, os.path.join(train_dir, cls))

    for img in val_imgs:
        shutil.copy(img, os.path.join(val_dir, cls))

print("✔ Train/val split completed successfully!")


✔ Train/val split completed successfully!


In [ ]:
!find /content/plantvillage_split/train -type f | head

/content/plantvillage_split/train/Tomato_Late_blight/e64d0e92-dbc2-44c2-b96a-3da5dc3a1c39___GHLB_PS Leaf 15.1 Day 8.jpg
/content/plantvillage_split/train/Tomato_Late_blight/10020ad2-c9bc-448b-a2de-7a8d34796f96___GHLB_PS Leaf 2.1 Day 13.jpg
/content/plantvillage_split/train/Tomato_Late_blight/4b115bc3-911c-4e98-b948-a6a360f06a87___GHLB_PS Leaf 25 Day 9.jpg
/content/plantvillage_split/train/Tomato_Late_blight/993825d6-dbbd-41b8-9993-c68c3504eff8___GHLB_PS Leaf 36 Day 13.jpg
/content/plantvillage_split/train/Tomato_Late_blight/452dfdfb-298d-4c0c-b500-0c480eea587d___GHLB_PS Leaf 40 Day 18.jpg
/content/plantvillage_split/train/Tomato_Late_blight/736cd652-e63d-4b01-a3c6-6859bc095fb9___GHLB_PS Leaf 39 Day 16.jpg
/content/plantvillage_split/train/Tomato_Late_blight/09cdb8a4-0c20-4acd-b4c8-709fad7010fd___GHLB_PS Leaf 37 Day 16.jpg
/content/plantvillage_split/train/Tomato_Late_blight/670616fe-c1a6-41a9-99b2-eb6270d8c299___GHLB_PS Leaf 26.2 Day 16.jpg
/content/plantvillage_split/train/Tomato_Late

In [ ]:
!ls -R /content/plantvillage_split/train | head -n 50


/content/plantvillage_split/train:
Pepper__bell___Bacterial_spot
Pepper__bell___healthy
Potato___Early_blight
Potato___healthy
Potato___Late_blight
Tomato_Bacterial_spot
Tomato_Early_blight
Tomato_healthy
Tomato_Late_blight
Tomato_Leaf_Mold
Tomato_Septoria_leaf_spot
Tomato_Spider_mites_Two_spotted_spider_mite
Tomato__Target_Spot
Tomato__Tomato_mosaic_virus
Tomato__Tomato_YellowLeaf__Curl_Virus

/content/plantvillage_split/train/Pepper__bell___Bacterial_spot:

/content/plantvillage_split/train/Pepper__bell___healthy:

/content/plantvillage_split/train/Potato___Early_blight:

/content/plantvillage_split/train/Potato___healthy:

/content/plantvillage_split/train/Potato___Late_blight:

/content/plantvillage_split/train/Tomato_Bacterial_spot:

/content/plantvillage_split/train/Tomato_Early_blight:

/content/plantvillage_split/train/Tomato_healthy:

/content/plantvillage_split/train/Tomato_Late_blight:
0172b2ff-6f6f-43e1-9a63-ea4bb2bce450___GHLB_PS Leaf 17 Day 9.jpg
07b42bb5-9f6c-43bc-9bbe-e

In [ ]:
!find /content/plantvillage_split/train -type f | wc -l

127


In [ ]:
import os

base = "/content/plantvillage_split"

def rename_files(split):
    root = os.path.join(base, split)
    for cls in os.listdir(root):
        cls_path = os.path.join(root, cls)
        if os.path.isdir(cls_path):
            for f in os.listdir(cls_path):
                old = os.path.join(cls_path, f)
                # convert filename to lowercase and replace spaces
                new = os.path.join(cls_path, f.lower().replace(" ", "_"))
                try:
                    os.rename(old, new)
                except:
                    pass

rename_files("train")
rename_files("val")



train_dir


'/content/plantvillage_split/train'

In [ ]:
!find /content/plantvillage_split/train -maxdepth 2 -type f | head -n 20




/content/plantvillage_split/train/Tomato_Late_blight/302da4a0-3b99-46ad-80f6-16ae585abf24___ghlb_ps_leaf_2.1_day_18.jpg
/content/plantvillage_split/train/Tomato_Late_blight/ed974f5b-e999-46a9-9e22-da51da2766b6___ghlb_ps_leaf_8_day_16.jpg
/content/plantvillage_split/train/Tomato_Late_blight/fb5dc387-cb1d-49bb-b81e-58d60705a2fd___ghlb_ps_leaf_1.3_day_16.jpg
/content/plantvillage_split/train/Tomato_Late_blight/bd9341f4-34d9-4acb-9aa0-5dbff08f8ba6___ghlb_ps_leaf_23_day_9.jpg
/content/plantvillage_split/train/Tomato_Late_blight/736cd652-e63d-4b01-a3c6-6859bc095fb9___ghlb_ps_leaf_39_day_16.jpg
/content/plantvillage_split/train/Tomato_Late_blight/bff1cbac-a141-4eda-ab29-b34dee705bce___ghlb_ps_leaf_47_day_18.jpg
/content/plantvillage_split/train/Tomato_Late_blight/a5ef7726-d20f-4109-985f-7fc8989e4aa5___ghlb_ps_leaf_2.2_day_8.jpg
/content/plantvillage_split/train/Tomato_Late_blight/121097dd-b0cb-436b-9b2f-7dab30c86872___ghlb_ps_leaf_37.1_day_13.jpg
/content/plantvillage_split/train/Tomato_Late_

In [ ]:
import os

root = "/content/plantvillage_split/train"

for path, subdirs, files in os.walk(root):
    for file in files:
        old = os.path.join(path, file)
        new = os.path.join(path, file.lower())
        if old != new:
            os.rename(old, new)



In [ ]:
!for d in /content/plantvillage_split/val/*; do mv "$d" "$(echo $d | tr 'A-Z' 'a-z')"; done


In [ ]:
import os

root = "/content/plantvillage_split/train"

for folder in os.listdir(root):
    full = os.path.join(root, folder)
    if os.path.isdir(full) and len(os.listdir(full)) == 0:
        print("EMPTY:", full)


EMPTY: /content/plantvillage_split/train/Tomato__Tomato_mosaic_virus
EMPTY: /content/plantvillage_split/train/Potato___Early_blight
EMPTY: /content/plantvillage_split/train/Tomato_healthy
EMPTY: /content/plantvillage_split/train/Tomato_Septoria_leaf_spot
EMPTY: /content/plantvillage_split/train/Tomato__Tomato_YellowLeaf__Curl_Virus
EMPTY: /content/plantvillage_split/train/Potato___healthy
EMPTY: /content/plantvillage_split/train/Tomato_Leaf_Mold
EMPTY: /content/plantvillage_split/train/Tomato__Target_Spot
EMPTY: /content/plantvillage_split/train/Tomato_Early_blight
EMPTY: /content/plantvillage_split/train/Potato___Late_blight
EMPTY: /content/plantvillage_split/train/Tomato_Spider_mites_Two_spotted_spider_mite
EMPTY: /content/plantvillage_split/train/Pepper__bell___healthy
EMPTY: /content/plantvillage_split/train/Tomato_Bacterial_spot
EMPTY: /content/plantvillage_split/train/Pepper__bell___Bacterial_spot


In [ ]:
import os, shutil

root = "/content/plantvillage_split/train"

for folder in os.listdir(root):
    full = os.path.join(root, folder)
    if os.path.isdir(full) and len(os.listdir(full)) == 0:
        print("Deleting:", full)
        shutil.rmtree(full)

Deleting: /content/plantvillage_split/train/Tomato__Tomato_mosaic_virus
Deleting: /content/plantvillage_split/train/Potato___Early_blight
Deleting: /content/plantvillage_split/train/Tomato_healthy
Deleting: /content/plantvillage_split/train/Tomato_Septoria_leaf_spot
Deleting: /content/plantvillage_split/train/Tomato__Tomato_YellowLeaf__Curl_Virus
Deleting: /content/plantvillage_split/train/Potato___healthy
Deleting: /content/plantvillage_split/train/Tomato_Leaf_Mold
Deleting: /content/plantvillage_split/train/Tomato__Target_Spot
Deleting: /content/plantvillage_split/train/Tomato_Early_blight
Deleting: /content/plantvillage_split/train/Potato___Late_blight
Deleting: /content/plantvillage_split/train/Tomato_Spider_mites_Two_spotted_spider_mite
Deleting: /content/plantvillage_split/train/Pepper__bell___healthy
Deleting: /content/plantvillage_split/train/Tomato_Bacterial_spot
Deleting: /content/plantvillage_split/train/Pepper__bell___Bacterial_spot


In [ ]:
import os

print("Train dir exists:", os.path.exists(train_dir))
print("Val dir exists:", os.path.exists(val_dir))

print("Train folders:", os.listdir(train_dir))
print("Val folders:", os.listdir(val_dir))


Train dir exists: True
Val dir exists: True
Train folders: ['Tomato_Late_blight']
Val folders: ['tomato__target_spot', 'tomato_early_blight', 'potato___early_blight', 'tomato_spider_mites_two_spotted_spider_mite', 'pepper__bell___bacterial_spot', 'potato___healthy', 'tomato__tomato_yellowleaf__curl_virus', 'tomato_septoria_leaf_spot', 'tomato_healthy', 'potato___late_blight', 'tomato_leaf_mold', 'tomato_late_blight', 'tomato_bacterial_spot', 'pepper__bell___healthy', 'tomato__tomato_mosaic_virus']


In [ ]:
!pip install streamlit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 126.3 MB/s eta 0:00:00


In [ ]:
import streamlit as st
print("Streamlit installed successfully!")


Streamlit installed successfully!


In [ ]:
from torchvision import models
import torch.nn as nn

# Number of classes
num_classes = len(class_names)

# Load pre-trained ResNet50
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)

# Replace the final layer
model.fc = nn.Linear(model.fc.in_features, num_classes)


In [ ]:
model_path = "/content/PlantDocBot/models/image_model.pth"
model.load_state_dict(torch.load(model_path, map_location="cpu"))

# Set model to evaluation mode
model.eval()

print("✅ Model loaded successfully!")


✅ Model loaded successfully!


In [ ]:
from PIL import Image
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

def predict_image(image):
    image = image.convert("RGB")
    img_tensor = transform(image).unsqueeze(0)

    with torch.no_grad():
        outputs = model(img_tensor)
        _, predicted = torch.max(outputs, 1)

    return class_names[predicted.item()]


In [ ]:
from google.colab import files
uploaded = files.upload()

for filename in uploaded.keys():
    image = Image.open(filename)
    prediction = predict_image(image)
    print("🌿 Prediction:", prediction)


Saving istockphoto-496662675-1024x1024.jpg to istockphoto-496662675-1024x1024 (3).jpg
🌿 Prediction: Tomato_Late_blight


In [ ]:
import streamlit as st
import torch
from torchvision import models, transforms
from PIL import Image
import os

# Load class names
train_dir = "/content/PlantDocBot/data/train"
class_names = sorted(os.listdir(train_dir))

# Create model architecture
num_classes = len(class_names)
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)

# Load saved weights
model_path = "/content/PlantDocBot/models/image_model.pth"
model.load_state_dict(torch.load(model_path, map_location="cpu"))
model.eval()

# Image transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])


In [ ]:
def predict_image(image):
    image = image.convert("RGB")
    img_tensor = transform(image).unsqueeze(0)

    with torch.no_grad():
        outputs = model(img_tensor)
        _, predicted = torch.max(outputs, 1)

    return class_names[predicted.item()]


In [ ]:
uploaded_file = st.file_uploader("📥 Upload a leaf image", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    image = Image.open(uploaded_file)
    st.image(image, caption="Uploaded Leaf Image", use_column_width=True)

    # Predict disease
    prediction = predict_image(image)
    st.success(f"🌿 Predicted Disease: **{prediction}**")


2025-12-12 12:30:37.589 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-12 12:30:37.590 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-12 12:30:37.591 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-12 12:30:37.604 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-12 12:30:37.656 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-12-12 12:30:37.657 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-12 12:30:37.658 Thread 'MainThread'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

BASE_DIR = '/content/drive/MyDrive/PlantBot'

from google.colab import files
files.upload()


Mounted at /content/drive


{}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"radhikapingale","key":"c0419da54f0724e915234908ed1506f5"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!ls ~/.kaggle


kaggle.json


In [ ]:
!kaggle datasets download -d emmarex/plantdisease -p /content


Dataset URL: https://www.kaggle.com/datasets/emmarex/plantdisease
License(s): unknown
 98% 645M/658M [00:07<00:00, 152MB/s]
100% 658M/658M [00:07<00:00, 97.8MB/s]


In [ ]:
!unzip -q /content/plantdisease.zip -d /content/plantvillage


In [ ]:
!ls /content/plantvillage/plantvillage/PlantVillage | head


Pepper__bell___Bacterial_spot
Pepper__bell___healthy
Potato___Early_blight
Potato___healthy
Potato___Late_blight
Tomato_Bacterial_spot
Tomato_Early_blight
Tomato_healthy
Tomato_Late_blight
Tomato_Leaf_Mold


In [ ]:
import os, shutil, random
from glob import glob

src_root = "/content/plantvillage/plantvillage/PlantVillage"  # Correct path
dst_root = "/content/plantvillage_split"

train_dir = os.path.join(dst_root, "train")
val_dir = os.path.join(dst_root, "val")

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

classes = os.listdir(src_root)
split_ratio = 0.8  # 80% train / 20% val

for cls in classes:
    src_cls = os.path.join(src_root, cls)
    images = glob(os.path.join(src_cls, "*.jpg"))

    random.shuffle(images)
    split = int(len(images) * split_ratio)  # <--- fix here
    train_imgs = images[:split]
    val_imgs = images[split:]

    os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(val_dir, cls), exist_ok=True)

    for img in train_imgs:
        shutil.copy(img, os.path.join(train_dir, cls))

    for img in val_imgs:
        shutil.copy(img, os.path.join(val_dir, cls))

print("✔ Train/val split completed successfully!")


✔ Train/val split completed successfully!


In [ ]:
!ls /content/plantvillage_split/train | head
!ls /content/plantvillage_split/val | head


Pepper__bell___Bacterial_spot
Pepper__bell___healthy
Potato___Early_blight
Potato___healthy
Potato___Late_blight
Tomato_Bacterial_spot
Tomato_Early_blight
Tomato_healthy
Tomato_Late_blight
Tomato_Leaf_Mold
Pepper__bell___Bacterial_spot
Pepper__bell___healthy
Potato___Early_blight
Potato___healthy
Potato___Late_blight
Tomato_Bacterial_spot
Tomato_Early_blight
Tomato_healthy
Tomato_Late_blight
Tomato_Leaf_Mold


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

# Use GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


In [ ]:
!ls /content/plantvillage/plantvillage/PlantVillage | head


Pepper__bell___Bacterial_spot
Pepper__bell___healthy
Potato___Early_blight
Potato___healthy
Potato___Late_blight
Tomato_Bacterial_spot
Tomato_Early_blight
Tomato_healthy
Tomato_Late_blight
Tomato_Leaf_Mold


In [ ]:
!ls /content/plantvillage/plantvillage/PlantVillage/Pepper__bell___Bacterial_spot | head


0022d6b7-d47c-4ee2-ae9a-392a53f48647___JR_B.Spot 8964.JPG
006adb74-934f-448f-a14f-62181742127b___JR_B.Spot 3395.JPG
00f2e69a-1e56-412d-8a79-fdce794a17e4___JR_B.Spot 3132.JPG
01613cd0-d3cd-4e96-945c-a312002037bf___JR_B.Spot 3262.JPG
0169b9ac-07b9-4be1-8b85-da94481f05a4___NREC_B.Spot 9169.JPG
018e494e-d2eb-468b-9d02-40219d9f4921___JR_B.Spot 9045.JPG
01940b6d-7dea-4889-a7b8-a35f4e9bba34___NREC_B.Spot 9120.JPG
01dfb88b-cd5a-420c-b163-51f5fe07b74d___JR_B.Spot 9091.JPG
01ebc916-4793-40a3-b5e4-a32687e4fa3d___NREC_B.Spot 9125.JPG
024623ab-be81-4d99-a653-c3be1495ce8e___JR_B.Spot 8889.JPG


In [ ]:
src_root = "/content/plantvillage/plantvillage/PlantVillage"  # source images
dst_root = "/content/plantvillage_split"                      # destination train/val folders


In [ ]:
import os, shutil, random
from glob import glob

src_root = "/content/plantvillage/plantvillage/PlantVillage"  # source
dst_root = "/content/plantvillage_split"                      # destination

train_dir = os.path.join(dst_root, "train")
val_dir   = os.path.join(dst_root, "val")
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

classes = os.listdir(src_root)
split_ratio = 0.8

for cls in classes:
    src_cls = os.path.join(src_root, cls)
    images = glob(os.path.join(src_cls, "*.jpg"))  # only .jpg files
    if len(images) == 0:
        print("No images found in:", src_cls)
        continue

    random.shuffle(images)
    split = int(len(images) * split_ratio)
    train_imgs = images[:split]
    val_imgs   = images[split:]

    os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(val_dir, cls), exist_ok=True)

    for img in train_imgs:
        shutil.copy(img, os.path.join(train_dir, cls))
    for img in val_imgs:
        shutil.copy(img, os.path.join(val_dir, cls))

print("✔ Train/val split completed successfully!")


No images found in: /content/plantvillage/plantvillage/PlantVillage/Tomato_Bacterial_spot
No images found in: /content/plantvillage/plantvillage/PlantVillage/Potato___Early_blight
No images found in: /content/plantvillage/plantvillage/PlantVillage/Tomato_Spider_mites_Two_spotted_spider_mite
No images found in: /content/plantvillage/plantvillage/PlantVillage/Tomato__Target_Spot
No images found in: /content/plantvillage/plantvillage/PlantVillage/Tomato_Early_blight
No images found in: /content/plantvillage/plantvillage/PlantVillage/Potato___healthy
No images found in: /content/plantvillage/plantvillage/PlantVillage/Potato___Late_blight
No images found in: /content/plantvillage/plantvillage/PlantVillage/Tomato_Septoria_leaf_spot
No images found in: /content/plantvillage/plantvillage/PlantVillage/Pepper__bell___Bacterial_spot
No images found in: /content/plantvillage/plantvillage/PlantVillage/Tomato__Tomato_mosaic_virus
No images found in: /content/plantvillage/plantvillage/PlantVillage/To

In [ ]:
!rm -rf /content/plantvillage_split


In [ ]:
!ls /content/plantvillage


plantvillage  PlantVillage


In [ ]:
!ls /content/plantvillage/plantvillage/PlantVillage | head


Pepper__bell___Bacterial_spot
Pepper__bell___healthy
Potato___Early_blight
Potato___healthy
Potato___Late_blight
Tomato_Bacterial_spot
Tomato_Early_blight
Tomato_healthy
Tomato_Late_blight
Tomato_Leaf_Mold


In [ ]:
!ls /content/plantvillage/plantvillage/PlantVillage/Tomato_healthy | head


000146ff-92a4-4db6-90ad-8fce2ae4fddd___GH_HL Leaf 259.1.JPG
000bf685-b305-408b-91f4-37030f8e62db___GH_HL Leaf 308.1.JPG
0031da2a-8edd-468f-a8b1-106657717a32___RS_HL 0105.JPG
003944fc-3b99-4a0f-9ed4-0e07352fd8b3___RS_HL 9844.JPG
00bce074-967b-4d50-967a-31fdaa35e688___RS_HL 0223.JPG
014b5e19-7917-4d76-b632-b5dd31d999ec___RS_HL 9640.JPG
01700b46-3800-46c6-ab81-09a5f50af525___RS_HL 0108.JPG
0172e56c-8bb8-4e75-8ac7-509df81393e8___RS_HL 0580.JPG
017a4026-813a-4983-887a-4052bb78c397___RS_HL 0218.JPG
0184dc6b-bfc1-4515-ac59-bbadaf524e1a___GH_HL Leaf 422.1.JPG


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


In [ ]:
train_tfms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
])

val_tfms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])


In [ ]:
import os

train_root = "/content/plantvillage_split/train"
val_root = "/content/plantvillage_split/val"

for folder in [train_root, val_root]:
    print(f"\nFolder: {folder}")
    for cls in os.listdir(folder):
        cls_path = os.path.join(folder, cls)
        if os.path.isdir(cls_path):
            num_imgs = len(os.listdir(cls_path))
            print(f"{cls}: {num_imgs} images")



Folder: /content/plantvillage_split/train
Pepper__bell___healthy: 0 images
Tomato_Late_blight: 121 images
Tomato_healthy: 0 images

Folder: /content/plantvillage_split/val
Pepper__bell___healthy: 1 images
Tomato_Late_blight: 31 images
Tomato_healthy: 1 images


In [ ]:
!ls /content/plantvillage_data/plantvillage


ls: cannot access '/content/plantvillage_data/plantvillage': No such file or directory


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Correct way
dataset_path = "/content/drive/MyDrive/Datasets/PlantVillage"


In [ ]:
dataset_path = "/content/drive/MyDrive/ColabDatasets/PlantVillage"


In [ ]:
####  15-12


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

Using device: cuda


In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std= [0.229, 0.224, 0.225]
    )
])

In [ ]:
import os
from torchvision import datasets

train_dir = "/content/PlantDocBot/data/train"
val_dir = "/content/PlantDocBot/data/val"

# 🔍 Check existence
if not os.path.exists(train_dir) or not os.path.exists(val_dir):
    raise FileNotFoundError("❌ train/val directory does not exist")

print("Train folder contents:", os.listdir(train_dir))
print("Val folder contents:", os.listdir(val_dir))

# 🚫 Prevent ImageFolder crash if empty
if len(os.listdir(train_dir)) == 0 or len(os.listdir(val_dir)) == 0:
    raise RuntimeError(
        "❌ Train/Val folders are EMPTY.\n"
        "Run dataset split cell again before loading ImageFolder."
    )

# ✅ Now it is SAFE to load
train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
val_dataset = datasets.ImageFolder(val_dir, transform=val_transform)

print("✅ Train images:", len(train_dataset))
print("✅ Val images:", len(val_dataset))
print("✅ Classes:", len(train_dataset.classes))
print("Sample train classes:", train_dataset.classes[:5])

FileNotFoundError: ❌ train/val directory does not exist

In [ ]:
16-12-off
17-12 below


**SPLIT DATASET (MANDATORY – AGAIN)** there is one code see that in recording

In [ ]:


import os, shutil, random

raw_dir = "/content/plantvillage_data/plantvillage/PlantVillage"
train_dir = "/content/data/train"
val_dir = "/content/data/val"

split_ratio = 0.8

for cls in os.listdir(raw_dir):
    cls_path = os.path.join(raw_dir, cls)
    if not os.path.isdir(cls_path):
        continue

    images = os.listdir(cls_path)
    if len(images) == 0:
        continue

    random.shuffle(images)
    split = int(len(images) * split_ratio)

    os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(val_dir, cls), exist_ok=True)

    for img in images[:split]:
        shutil.copy(
            os.path.join(cls_path, img),
            os.path.join(train_dir, cls, img)
        )

    for img in images[split:]:
        shutil.copy(
            os.path.join(cls_path, img),
            os.path.join(val_dir, cls, img)
        )

print("✅ Dataset split completed")

VERIFY TRAIN / VAL CONTENTS
VERIFY TRAIN / VAL CONTENT **bold text**

In [ ]:


import os

train_dir = "/content/data/train"
val_dir = "/content/data/val"

print("Train folder exists:", os.path.exists(train_dir))
print("Val folder exists:", os.path.exists(val_dir))

print("Train classes:", os.listdir(train_dir))
print("Val classes:", os.listdir(val_dir))

In [ ]:
from torchvision import transforms

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

print("Transforms defined")

In [ ]:
from torchvision import datasets

train_dataset = datasets.ImageFolder(
    root=train_dir,
    transform=train_transform
)

val_dataset = datasets.ImageFolder(
    root=val_dir,
    transform=val_transform
)

print("Train images:", len(train_dataset))
print("Val images:", len(val_dataset))

**Save Model**+CLASS MODEL

In [ ]:
import torch

model_save_path = "/content/plant_disease_resnet50.pth"

torch.save({
    "model_state_dict": model.state_dict(),
    "class_names": train_dataset.classes
}, model_save_path)

print("✅ Model saved successfully at:", model_save_path)

NameError: name 'model' is not defined